# Data Cleaning 


In [ ]:
import pandas as pd
import numpy as np 
import os 

: 

In [ ]:
os.chdir('..')

from src.data_ingestion import data_ingestion
from src.utils.data_preprocessing import is_missing_value, remove_missing_values , remove_outliers , is_outlier , fill_missing_values_with_mean 
